In [3]:
import pandas as pd
import folium
from geopy.geocoders import Nominatim
from geopy.exc import GeocoderTimedOut
from docplex.mp.model import Model
import docplex.mp.solution as Solution

In [4]:
df = pd.read_excel("TSP Dataset/Dataset 1.xlsx", sheet_name=0)
df = df.set_index('İLÇE ADI') 
df.index.names = [None] 
df=df.fillna(0)

In [28]:
n = 16
cities =[i for i in range(n)] 
edges =[(i,j) for i in cities for j in cities if i!=j]

In [29]:
m=Model('TSP')

In [30]:
x = m.binary_var_dict(edges, name = 'x')
u = m.continuous_var_dict(cities, name ='u')

In [31]:
u

{0: docplex.mp.Var(type=C,name='u_0'),
 1: docplex.mp.Var(type=C,name='u_1'),
 2: docplex.mp.Var(type=C,name='u_2'),
 3: docplex.mp.Var(type=C,name='u_3'),
 4: docplex.mp.Var(type=C,name='u_4'),
 5: docplex.mp.Var(type=C,name='u_5'),
 6: docplex.mp.Var(type=C,name='u_6'),
 7: docplex.mp.Var(type=C,name='u_7'),
 8: docplex.mp.Var(type=C,name='u_8'),
 9: docplex.mp.Var(type=C,name='u_9'),
 10: docplex.mp.Var(type=C,name='u_10'),
 11: docplex.mp.Var(type=C,name='u_11'),
 12: docplex.mp.Var(type=C,name='u_12'),
 13: docplex.mp.Var(type=C,name='u_13'),
 14: docplex.mp.Var(type=C,name='u_14'),
 15: docplex.mp.Var(type=C,name='u_15')}

In [32]:
distances ={(i, j): df.iloc[i, j] for i,j in edges}

In [33]:
distances

{(0, 1): 59.03976799,
 (0, 2): 67.80542849000001,
 (0, 3): 49.36197077,
 (0, 4): 61.77098388,
 (0, 5): 62.14474955,
 (0, 6): 50.89906439,
 (0, 7): 47.36359723,
 (0, 8): 46.32952767,
 (0, 9): 74.7942733,
 (0, 10): 59.54967153,
 (0, 11): 43.64795046,
 (0, 12): 53.32529212,
 (0, 13): 35.9767483,
 (0, 14): 62.46433121000001,
 (0, 15): 49.69779077,
 (1, 0): 59.03976799,
 (1, 2): 43.63400693,
 (1, 3): 29.78076195,
 (1, 4): 29.77196755,
 (1, 5): 29.39820189,
 (1, 6): 37.12515776,
 (1, 7): 23.92569771,
 (1, 8): 12.71024032,
 (1, 9): 21.81576921,
 (1, 10): 50.7913873,
 (1, 11): 20.21005093,
 (1, 12): 57.01576671,
 (1, 13): 75.99955125,
 (1, 14): 9.485827129999999,
 (1, 15): 29.44494195,
 (2, 0): 67.80542849000001,
 (2, 1): 43.63400693,
 (2, 3): 22.14815464,
 (2, 4): 13.86203937,
 (2, 5): 14.23580504,
 (2, 6): 18.5063641,
 (2, 7): 19.70830922,
 (2, 8): 30.9237666,
 (2, 9): 59.38851223,
 (2, 10): 7.15738038,
 (2, 11): 23.42395599,
 (2, 12): 13.38175978,
 (2, 13): 32.36554432,
 (2, 14): 47.0585701

In [34]:
m.minimize(m.sum(distances[e]*x[e] for e in edges))

In [35]:
# Constraint 1: each city must be entered once
for c in cities:
    if c != 0:
        m.add_constraint(m.sum(x[(i,j)] for i,j in edges if i==c)==1, ctname=f'city_out__{c}')

# Constraint 2: each city must be exited once
for c in cities:
    if c != 0:
        m.add_constraint(m.sum(x[(i,j)] for i,j in edges if j==c)==1, ctname=f'city_in_{c}')

In [36]:
# Constraint 3: ensures that u_j = u_i + 1 if and only if x_ij = 1
for i,j in edges:
    if j!=0:
        m.add_indicator(x[(i,j)],u[(i)]+1==u[(j)], name='order_{i}_{j}')

In [37]:
print(m.export_to_string())

\ This file has been generated by DOcplex
\ ENCODING=ISO-8859-1
\Problem name: TSP

Minimize
 obj: 59.039767990000 x_0_1 + 67.805428490000 x_0_2 + 49.361970770000 x_0_3
      + 61.770983880000 x_0_4 + 62.144749550000 x_0_5 + 50.899064390000 x_0_6
      + 47.363597230000 x_0_7 + 46.329527670000 x_0_8 + 74.794273300000 x_0_9
      + 59.549671530000 x_0_10 + 43.647950460000 x_0_11 + 53.325292120000 x_0_12
      + 35.976748300000 x_0_13 + 62.464331210000 x_0_14 + 49.697790770000 x_0_15
      + 59.039767990000 x_1_0 + 43.634006930000 x_1_2 + 29.780761950000 x_1_3
      + 29.771967550000 x_1_4 + 29.398201890000 x_1_5 + 37.125157760000 x_1_6
      + 23.925697710000 x_1_7 + 12.710240320000 x_1_8 + 21.815769210000 x_1_9
      + 50.791387300000 x_1_10 + 20.210050930000 x_1_11 + 57.015766710000 x_1_12
      + 75.999551250000 x_1_13 + 9.485827130000 x_1_14 + 29.444941950000 x_1_15
      + 67.805428490000 x_2_0 + 43.634006930000 x_2_1 + 22.148154640000 x_2_3
      + 13.862039370000 x_2_4 + 14.23580

In [38]:
# m.parameters.timelimit=600
# m.parameters.mip.strategy.branch=1
# m.parameters.mip.tolerances.mipgap=0.15

solution = m.solve(log_output=True)

Version identifier: 22.1.1.0 | 2023-06-15 | d64d5bd77
CPXPARAM_Read_DataCheck                          1
Tried aggregator 2 times.
MIP Presolve modified 105 coefficients.
Aggregator did 105 substitutions.
Reduced MIP has 150 rows, 376 columns, and 810 nonzeros.
Reduced MIP has 240 binaries, 0 generals, 0 SOSs, and 225 indicators.
Presolve time = 0.00 sec. (0.72 ticks)
Found incumbent of value 1648.342295 after 0.00 sec. (1.05 ticks)
Probing time = 0.00 sec. (0.39 ticks)
Tried aggregator 1 time.
Detecting symmetries...
Reduced MIP has 150 rows, 376 columns, and 810 nonzeros.
Reduced MIP has 240 binaries, 0 generals, 0 SOSs, and 225 indicators.
Presolve time = 0.00 sec. (0.55 ticks)
Probing time = 0.00 sec. (0.39 ticks)
Clique table members: 135.
MIP emphasis: balance optimality and feasibility.
MIP search method: dynamic search.
Parallel mode: deterministic, using up to 8 threads.
Root relaxation solution time = 0.00 sec. (0.27 ticks)

        Nodes                                      

 2845947 264660      223.7809     4      237.7572      222.2678  4878201    6.51%
Elapsed time = 355.16 sec. (50796.77 ticks, tree = 93.62 MB, solutions = 17)
 2889570 260064      224.4843     6      237.7572      222.7379  4931944    6.32%
 2934256 255437      232.9357     7      237.7572      223.2828  5005014    6.09%
 2983903 250475    infeasible            237.7572      223.6771  5061687    5.92%
 3025325 240941    infeasible            237.7572      224.1782  5176497    5.71%
 3073328 236695      226.0523     6      237.7572      224.4843  5234289    5.58%
 3121062 228913    infeasible            237.7572      224.8877  5309556    5.41%
 3162798 219587      226.3338     6      237.7572      225.3034  5398347    5.24%
 3204917 214868      227.1520     4      237.7572      225.7788  5458246    5.04%
 3253104 203725    infeasible            237.7572      226.3338  5549355    4.80%
 3304332 193867      228.2072     4      237.7572      227.0352  5637211    4.51%
Elapsed time = 428.55

In [39]:
solution.display()

solution for: TSP
objective: 237.757
status: OPTIMAL_SOLUTION(2)
x_0_13 = 1
x_1_0 = 1
x_2_4 = 1
x_3_15 = 1
x_4_5 = 1
x_5_6 = 1
x_6_3 = 1
x_7_11 = 1
x_8_9 = 1
x_9_14 = 1
x_10_2 = 1
x_11_8 = 1
x_12_10 = 1
x_13_12 = 1
x_14_1 = 1
x_15_7 = 1
u_1 = 15.000
u_2 = 4.000
u_3 = 8.000
u_4 = 5.000
u_5 = 6.000
u_6 = 7.000
u_7 = 10.000
u_8 = 12.000
u_9 = 13.000
u_10 = 3.000
u_11 = 11.000
u_12 = 2.000
u_13 = 1.000
u_14 = 14.000
u_15 = 9.000


In [40]:
lst_c = []
for c in u:
    soln_c = (c,solution.get_var_value(u[c]))
    lst_c.append(soln_c)
df_c = pd.DataFrame.from_records(lst_c, columns = ['city', 'visit order'])
df_c.sort_values(by=['visit order'], inplace = True)
df_c

,city,visit order
0,0,0.0
13,13,1.0
12,12,2.0
10,10,3.0
2,2,4.0
4,4,5.0
5,5,6.0
6,6,7.0
3,3,8.0
15,15,9.0


In [ ]:
def get_coordinates(dataframe):
    
    
    
    

In [5]:





def mapper(order, coordinates):

    geolocator = Nominatim(user_agent="tsp_visualization")

    def coordinates(district):
        location = geolocator.geocode(f"{district}, Istanbul, Turkey")
        return (location.latitude, location.longitude)

    # Get coordinates for all districts
    coordinates = {district: coordinates(district) for district in dataframe.index}

    # Create a folium map centered around Istanbul
    istanbul = geolocator.geocode("Istanbul, Turkey")
    m = folium.Map(location=(istanbul.latitude, istanbul.longitude), zoom_start=13)

    # Map the district names to their indices
    district_indices = {i: district for i, district in enumerate(dataframe.index)}

    # Add numbered markers for each district in the connection order
    for i, district_index in enumerate(order):
        district_name = district_indices.get(district_index)
        coord = coordinates.get(district_name)
        if coord:
            folium.Marker(
                location=coord,
                popup=f"{i}: {district_name}",
                tooltip=f"{i}: {district_name}",
                icon=folium.DivIcon(
                    html=f"""
                    <div style="
                        font-size: 14pt;
                        font-weight: bold;
                        color: white;
                        background-color: black;
                        border-radius: 50%;
                        width: 24px;
                        height: 24px;
                        text-align: center;
                        line-height: 24px;
                    ">{i}</div>
                    """
                )
            ).add_to(m)

     # Add lines connecting the districts in the given order
    for i in range(len(order) - 1):
        start_index = order[i]
        end_index = order[i + 1]
        start_district = dataframe.index[start_index]
        end_district = dataframe.index[end_index]
        start_coord = coordinates.get(start_district)
        end_coord = coordinates.get(end_district)
        if start_coord and end_coord:
            folium.PolyLine(locations=[start_coord, end_coord], color='blue').add_to(m)

    return m


In [6]:
df = df.iloc[:20, :20]
order = [0, 13, 12, 10, 2, 4, 5, 6, 3, 15, 7, 11, 8, 9, 14, 1, 0]


mapper(order, df)


In [ ]:
    district_indices = {i: district for i, district in enumerate(df.index)}


In [ ]:
district_indices

In [11]:
district_indices[15]

'ESENLER'

In [5]:
geolocator = Nominatim(user_agent="tsp_visualization")

def coordinates(district):
        location = geolocator.geocode(f"{district}, Istanbul, Turkey")
        return (location.latitude, location.longitude)

    # Get coordinates for all districts
coordinates = {district: coordinates(district) for district in df.index}

GeocoderUnavailable: HTTPSConnectionPool(host='nominatim.openstreetmap.org', port=443): Max retries exceeded with url: /search?q=BEYO%C4%9ELU%2C+Istanbul%2C+Turkey&format=json&limit=1 (Caused by ReadTimeoutError("HTTPSConnectionPool(host='nominatim.openstreetmap.org', port=443): Read timed out. (read timeout=1)"))

In [ ]:
coordinates